# NLP: Introdução a regex e modelos de linguagem

Material referente ao curso da Alura disponível [aqui](https://cursos.alura.com.br/course/nlp-modelos-linguagem). O propósito final deste notebook é criar um detector de linguagem utilizando regex e modelos de linguagem.

Serão utilizadas expressões regulares para facilitar o reconhecimento de padrões no texto. Mais informações [aqui](https://regex101.com/).

## Preparando o ambiente

In [1]:
!pip install --user -U nltk --no-warn-script-location

In [59]:
import pandas as pd
import re
from nltk.util import bigrams
from nltk.lm.preprocessing import pad_both_ends, padded_everygram_pipeline
from sklearn.model_selection import train_test_split
from nltk.tokenize import WhitespaceTokenizer
from nltk.lm import MLE, NgramCounter, Laplace

## Carregando os dados

In [3]:
dados_portugues = pd.read_csv('dados/stackoverflow_portugues.csv')
dados_ingles = pd.read_csv('dados/stackoverflow_ingles.csv')
dados_espanhol = pd.read_csv('dados/stackoverflow_espanhol.csv')

In [4]:
dados_portugues.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191


## Utilizando Regex

In [5]:
questao_portugues = dados_portugues['Questão'][5]
questao_ingles = dados_ingles['Questão'][5]
questao_espanhol = dados_espanhol['Questão'][5]

Encontrando todas as tags HTML do texto

In [6]:
re.findall(r"<.*?>", questao_portugues)

['<p>',
 '</p>',
 '<pre>',
 '<code>',
 '</code>',
 '</pre>',
 '<p>',
 '<code>',
 '</code>',
 '<code>',
 '</code>',
 '</p>',
 '<pre>',
 '<code>',
 '</code>',
 '</pre>',
 '<p>',
 '<code>',
 '</code>',
 '<code>',
 '</code>',
 '</p>',
 '<p>',
 '</p>']

Removendo as tags HTML do texto

In [7]:
re.sub(r"<.*?>", '', questao_portugues)

"Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo:\n\n//----CONSULTA SQL----//\n$busca = mysql_query ('insert into Produtos (coluna) values(' . $valor . ')');\n\n\nLogo, digamos que o usuário usar a sentença: 1); DROP TABLE Produtos; para ao campo valor o comando ficaria: \n\ninsert into Produtos (coluna) values(1); DROP TABLE Produtos;\n\n\nEle vai inserir um novo registro cujo o campo coluna será 1 e logo em seguida ele vai deletar a tabela Produtos.\n\nComo posso melhorar meu código para prevenir essa situação?\n"

Uma forma de fazer isso sem ter que reescrever a string de regex toda a hora:

In [8]:
regex = re.compile(r"<.*?>")

In [9]:
regex.findall(questao_portugues)

['<p>',
 '</p>',
 '<pre>',
 '<code>',
 '</code>',
 '</pre>',
 '<p>',
 '<code>',
 '</code>',
 '<code>',
 '</code>',
 '</p>',
 '<pre>',
 '<code>',
 '</code>',
 '</pre>',
 '<p>',
 '<code>',
 '</code>',
 '<code>',
 '</code>',
 '</p>',
 '<p>',
 '</p>']

In [10]:
regex.sub('', questao_portugues)

"Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo:\n\n//----CONSULTA SQL----//\n$busca = mysql_query ('insert into Produtos (coluna) values(' . $valor . ')');\n\n\nLogo, digamos que o usuário usar a sentença: 1); DROP TABLE Produtos; para ao campo valor o comando ficaria: \n\ninsert into Produtos (coluna) values(1); DROP TABLE Produtos;\n\n\nEle vai inserir um novo registro cujo o campo coluna será 1 e logo em seguida ele vai deletar a tabela Produtos.\n\nComo posso melhorar meu código para prevenir essa situação?\n"

In [11]:
def remover(textos, regex, sub=''):
    if type(textos) == str:
        return regex.sub(sub, textos)
    return [regex.sub(sub, texto) for texto in textos]

In [12]:
regex_html = re.compile(r"<.*?>")
remover(questao_portugues, regex_html)

"Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo:\n\n//----CONSULTA SQL----//\n$busca = mysql_query ('insert into Produtos (coluna) values(' . $valor . ')');\n\n\nLogo, digamos que o usuário usar a sentença: 1); DROP TABLE Produtos; para ao campo valor o comando ficaria: \n\ninsert into Produtos (coluna) values(1); DROP TABLE Produtos;\n\n\nEle vai inserir um novo registro cujo o campo coluna será 1 e logo em seguida ele vai deletar a tabela Produtos.\n\nComo posso melhorar meu código para prevenir essa situação?\n"

In [13]:
regex_code = re.compile(r"<code>(.|(\n))*?</code>")
regex_code.findall(questao_portugues)

[('\n', '\n'), (';', ''), ('r', ''), ('\n', '\n'), ('a', ''), ('1', '')]

In [14]:
remover(questao_portugues, regex_code)

'<p>Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo:</p>\n\n<pre></pre>\n\n<p>Logo, digamos que o usuário usar a sentença:  para ao campo  o comando ficaria: </p>\n\n<pre></pre>\n\n<p>Ele vai inserir um novo registro cujo o campo  será  e logo em seguida ele vai deletar a tabela Produtos.</p>\n\n<p>Como posso melhorar meu código para prevenir essa situação?</p>\n'

In [15]:
remover(questao_ingles, regex_code)

'<p>What is the use of the  keyword in Python? What does it do?</p>\n\n<p>For example, I\'m trying to understand this code<sup><strong>1</strong></sup>:</p>\n\n<pre></pre>\n\n<p>And this is the caller:</p>\n\n<pre></pre>\n\n<p>What happens when the method  is called?\nIs a list returned? A single element? Is it called again? When will subsequent calls stop?</p>\n\n<hr>\n\n<p><sub>\n1. This piece of code was written by Jochen Schulz (jrschulz), who made a great Python library for metric spaces. This is the link to the complete source: <a href="http://well-adjusted.de/~jrschulz/mspace/" rel="noreferrer">Module mspace</a>.</sub></p>\n'

In [16]:
remover(questao_espanhol, regex_code)

'<p>Siempre he visto que en  hay:</p>\n\n<ul>\n<li>asignaciones </li>\n<li>comparaciones  y </li>\n</ul>\n\n<p>Creo entender que  hace algo parecido a comparar el valor de la variable y el  también compara el tipo (como un equals de java). </p>\n\n<p><strong>¿Alguien podría confirmarme este punto y extenderlo?</strong>. Soy javero y el no tipado de javascript a veces me encanta y otras lo odio.</p>\n\n<hr>\n\n<p>¿Cuál es la manera correcta en javascript de comparar ,  y otros valores por defecto? </p>\n\n<pre></pre>\n\n<p>¿<strong></strong> se usa como cadena de texto o como palabra clave? ¿Cual de las siguientes comparaciones es la correcta para un elemento  sin ? (por ejemplo un label sin contenido)</p>\n\n<pre></pre>\n'

### Resumo

Remove os códigos:

In [17]:
regex_code = re.compile(r"<code>(.|(\n))*?</code>")
ret1 = remover(questao_ingles, regex_code)
ret1

'<p>What is the use of the  keyword in Python? What does it do?</p>\n\n<p>For example, I\'m trying to understand this code<sup><strong>1</strong></sup>:</p>\n\n<pre></pre>\n\n<p>And this is the caller:</p>\n\n<pre></pre>\n\n<p>What happens when the method  is called?\nIs a list returned? A single element? Is it called again? When will subsequent calls stop?</p>\n\n<hr>\n\n<p><sub>\n1. This piece of code was written by Jochen Schulz (jrschulz), who made a great Python library for metric spaces. This is the link to the complete source: <a href="http://well-adjusted.de/~jrschulz/mspace/" rel="noreferrer">Module mspace</a>.</sub></p>\n'

Remove o `\n`

In [18]:
regex_line_break = re.compile(r"\n")
ret2 = remover(ret1, regex_line_break)
ret2

'<p>What is the use of the  keyword in Python? What does it do?</p><p>For example, I\'m trying to understand this code<sup><strong>1</strong></sup>:</p><pre></pre><p>And this is the caller:</p><pre></pre><p>What happens when the method  is called?Is a list returned? A single element? Is it called again? When will subsequent calls stop?</p><hr><p><sub>1. This piece of code was written by Jochen Schulz (jrschulz), who made a great Python library for metric spaces. This is the link to the complete source: <a href="http://well-adjusted.de/~jrschulz/mspace/" rel="noreferrer">Module mspace</a>.</sub></p>'

Remove as tags:

In [19]:
regex_html_tags = re.compile(r"<.*?>")
ret3 = remover(ret2, regex_html_tags)
ret3

"What is the use of the  keyword in Python? What does it do?For example, I'm trying to understand this code1:And this is the caller:What happens when the method  is called?Is a list returned? A single element? Is it called again? When will subsequent calls stop?1. This piece of code was written by Jochen Schulz (jrschulz), who made a great Python library for metric spaces. This is the link to the complete source: Module mspace."

In [20]:
def tratar_texto(texto):
    regex_code = re.compile(r"<code>(.|(\n))*?</code>")
    regex_line_break = re.compile(r"\n")
    regex_html_tags = re.compile(r"<.*?>")
    
    texto = remover(texto, regex_code)
    texto = remover(texto, regex_line_break, sub=' ')
    texto = remover(texto, regex_html_tags)
    
    return texto

In [21]:
dados_portugues['t_questao'] = [tratar_texto(questao) for questao in dados_portugues['Questão']]
dados_ingles['t_questao'] = [tratar_texto(questao) for questao in dados_ingles['Questão']]
dados_espanhol['t_questao'] = [tratar_texto(questao) for questao in dados_espanhol['Questão']]

In [22]:
dados_portugues.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações,t_questao
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367,Se eu fizer o hash de senhas antes de armazená...
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953,Qual é a diferença entre e ? Podem me dar alg...
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761,Uma dúvida muito comum é por que devemos parar...
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075,É comum encontrar uma mensagem de erro que diz...
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191,Me parecem termos muito próximos e eventualmen...


### Removendo o que não é alfanumérico

Considerando o caracter especial `\w`

In [23]:
regex_pontuacao = re.compile(r'[^\w\s]')
regex_pontuacao.findall(questao_espanhol)

['<',
 '>',
 '<',
 '>',
 '<',
 '/',
 '>',
 ':',
 '<',
 '/',
 '>',
 '<',
 '>',
 '<',
 '>',
 '<',
 '>',
 '=',
 '<',
 '/',
 '>',
 '<',
 '/',
 '>',
 '<',
 '>',
 '<',
 '>',
 '=',
 '=',
 '<',
 '/',
 '>',
 '<',
 '>',
 '=',
 '=',
 '=',
 '<',
 '/',
 '>',
 '<',
 '/',
 '>',
 '<',
 '/',
 '>',
 '<',
 '>',
 '<',
 '>',
 '=',
 '=',
 '<',
 '/',
 '>',
 '<',
 '>',
 '=',
 '=',
 '=',
 '<',
 '/',
 '>',
 '(',
 ')',
 '.',
 '<',
 '/',
 '>',
 '<',
 '>',
 '<',
 '>',
 '¿',
 '?',
 '<',
 '/',
 '>',
 '.',
 '.',
 '<',
 '/',
 '>',
 '<',
 '>',
 '<',
 '>',
 '¿',
 '<',
 '>',
 '<',
 '/',
 '>',
 ',',
 '<',
 '>',
 '<',
 '/',
 '>',
 '?',
 '<',
 '/',
 '>',
 '<',
 '>',
 '<',
 '>',
 '=',
 '=',
 '=',
 '=',
 '=',
 '<',
 '/',
 '>',
 '<',
 '/',
 '>',
 '<',
 '>',
 '¿',
 '<',
 '>',
 '<',
 '>',
 '<',
 '/',
 '>',
 '<',
 '/',
 '>',
 '?',
 '¿',
 '<',
 '>',
 '<',
 '/',
 '>',
 '<',
 '>',
 '<',
 '/',
 '>',
 '?',
 '(',
 ')',
 '<',
 '/',
 '>',
 '<',
 '>',
 '<',
 '>',
 '=',
 '=',
 '"',
 '"',
 '=',
 '=',
 '=',
 '"',
 '"',
 '=',
 '=',
 '=',
 '='

In [24]:
def tratar_texto(texto):
    regex_code = re.compile(r"<code>(.|(\n))*?</code>")
    regex_line_break = re.compile(r"(\n)")
    regex_html_tags = re.compile(r"<.*?>")
    regex_pontuacao = re.compile(r'[^\w\s]')
    regex_numeros = re.compile(r'\d+')
    regex_spaces = re.compile(r'  ')
    
    texto = remover(texto, regex_code, sub=' ')
    texto = remover(texto, regex_line_break, sub=' ')
    texto = remover(texto, regex_html_tags, sub=' ')
    texto = remover(texto, regex_pontuacao, sub=' ')
    texto = remover(texto, regex_numeros, sub=' ')
    texto = remover(texto, regex_spaces)
    
    return texto.lower()

In [25]:
dados_portugues['t_questao'] = [tratar_texto(questao) for questao in dados_portugues['Questão']]
dados_ingles['t_questao'] = [tratar_texto(questao) for questao in dados_ingles['Questão']]
dados_espanhol['t_questao'] = [tratar_texto(questao) for questao in dados_espanhol['Questão']]

In [26]:
dados_espanhol['t_questao'][5]

' siempre he visto que en hay asignaciones comparaciones ycreo entender que hace algo parecido a comparar el valor de la variable y el también compara el tipocomo un equals de java alguien podría confirmarme este punto y extenderlosoy javero y el no tipado de javascript a veces me encanta y otras lo odio cuál es la manera correcta en javascript de comparary otros valores por defectose usa como cadena de texto o como palabra clave cual de las siguientes comparaciones es la correcta para un elemento sin por ejemplo un label sin contenido'

In [27]:
dados_espanhol['Questão'][5]

'<p>Siempre he visto que en <code>JavaScript</code> hay:</p>\n\n<ul>\n<li>asignaciones <code>=</code></li>\n<li>comparaciones <code>==</code> y <code>===</code></li>\n</ul>\n\n<p>Creo entender que <code>==</code> hace algo parecido a comparar el valor de la variable y el <code>===</code> también compara el tipo (como un equals de java). </p>\n\n<p><strong>¿Alguien podría confirmarme este punto y extenderlo?</strong>. Soy javero y el no tipado de javascript a veces me encanta y otras lo odio.</p>\n\n<hr>\n\n<p>¿Cuál es la manera correcta en javascript de comparar <code>undefined</code>, <code>null</code> y otros valores por defecto? </p>\n\n<pre><code>variable == null\nvariable === null\n</code></pre>\n\n<p>¿<strong><code>undefined</code></strong> se usa como cadena de texto o como palabra clave? ¿Cual de las siguientes comparaciones es la correcta para un elemento <code>html</code> sin <code>value</code>? (por ejemplo un label sin contenido)</p>\n\n<pre><code>variable == "undefined"\nv

## Utilizando NLTK

### `bigrams` e `pad_both_ends`

In [28]:
texto_teste = "salatiel"
list(bigrams(pad_both_ends(texto_teste, n = 2)))

[('<s>', 's'),
 ('s', 'a'),
 ('a', 'l'),
 ('l', 'a'),
 ('a', 't'),
 ('t', 'i'),
 ('i', 'e'),
 ('e', 'l'),
 ('l', '</s>')]

A ideia do `pad_both_ends` é evitar erro de probabilidade por os caracteres iniciais e finais aparecerem apenas uma vez enquanto os caracteres internos do texto sempre são duplicados

## Criando coluna idioma

In [29]:
dados_portugues['idioma'] = 'port'
dados_ingles['idioma'] = 'ing'
dados_espanhol['idioma'] = 'esp'

In [30]:
df_full = pd.concat([dados_portugues, dados_ingles, dados_espanhol])[['Id', 't_questao', 'idioma']]
X = df_full['t_questao']
y = df_full['idioma']

## Separando treino e teste

No curso o professor criou os dados de treino e teste individuais para cada DataFrame, o que não parece fazer muito sentido, então agrupei todos juntos e estou fazendo a separação junto proporcional pelo idioma.

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## Pipeline treinamento ML

In [32]:
dic_treino = { 'questao': X_train.values, 'idioma': y_train.values}
df_treino = pd.DataFrame(dic_treino)
pt_treino = df_treino.query("idioma == 'port'")
todas_questoes_port = ' '.join(pt_treino['questao'])
todas_questoes_port

'wikipedia sistema de computadores interligados em rede através da internetque cooperam de modo transparente para fornecer conteúdo cdn são geralmente implantados em vários locaisos benefícios incluem a redução dos custos de largura de bandamelhorando os tempos de carregamento da páginaou aumentando disponibilidade global de conteúdo baboo uma cdn é uma rede de distribuição de informação e dados que permite fornecer conteúdo na web de forma rápida para vários usuáriosindependentemente de onde essa pessoa se encontrao que o sistema faz é distribuir um mesmo conteúdo em diversos servidorespara direcionar esses dados de acordo com a proximidade de servidor do usuário o serviço funciona como um cache dos arquivos não dinâmicos do siteem diferentes datacenters de localidades dispersas em todo o mundo stackoverflow cdn é simplesmente uma rede de servidores que replicam seus arquivos binários de modo que eles são servidos a partir de locais geograficamente próximoscdn é simplesmente um bando 

In [33]:
dic_teste = { 'questao': X_test.values, 'idioma': y_test.values}
df_teste = pd.DataFrame(dic_teste)
pt_teste = df_teste.query("idioma == 'port'")
en_teste = df_teste.query("idioma == 'ing'")
es_teste = df_teste.query("idioma == 'esp'")

In [34]:
es_treino = df_treino.query("idioma == 'esp'")
todas_questoes_esp = ' '.join(es_treino['questao'])
todas_questoes_esp

' he buscado por internet pero no me ha quedado muy claro lo que significan estas palabras reservadas y qué funcionalidades tienen en una clasestaticfinal  esta respuestaen inglés explica cómo saber si tu javascript se está ejecutando en node js o en el navegadorpero depende del uso deque no funciona en un navegador si usas un csp siny eso es importante en mi proyectoyo sé que hay varias formas sensiblespero falibles también como puede ser ver si o están definidos pero es fácil definir estas variables para engañar al programahay alguna otra forma que sea infalible de detectar si estoy en el navegador o en node js sin usar supongo que por lo menos puedo intentar eso en un ysi estoy en node jspuedo estar con certezay si no puedo usar una forma menos infalible   ayer escribí por error y no me he dado cuenta porque el compilador no ha dado errormás tarde me he dado cuenta y me ha sorprendido he probado con este pequeño programa el cual funciona como es de esperar también funciona si pongo 

In [35]:
in_treino = df_treino.query("idioma == 'ing'")
todas_questoes_ing = ' '.join(in_treino['questao'])
todas_questoes_ing

' as joel points out instack overflow podcastinc programming language akakampr there is mention of this property of arrays in c joel says that it s because of pointer arithmetic but i still don t understand why does  my initial commit contained some log filesi ve added to myand now i want to remove the log files from my repositorywill remove a file from the repositorybut will also remove it from the local file system how can i remove this file from the repowithoutdeleting my local copy of the file   how come certain random strings produce colors when entered as background colors in htmlfor example\r\r\r\r\rproduces a document with ared backgroundacross all browsers and platforms interestinglywhile produces a red background as wellproduces a yellow background what s going on here   what is this idiom and when should it be usedwhich problems does it solvedoes the idiom change when cis used although it s been mentioned in many placeswe didn t have any singularwhat is itquestion and answer

## Tokenização

In [36]:
tokenizer = WhitespaceTokenizer()
todas_palavras_port = tokenizer.tokenize(todas_questoes_port)
todas_palavras_esp = tokenizer.tokenize(todas_questoes_esp)
todas_palavras_ing = tokenizer.tokenize(todas_questoes_ing)
todas_palavras_port

['wikipedia',
 'sistema',
 'de',
 'computadores',
 'interligados',
 'em',
 'rede',
 'através',
 'da',
 'internetque',
 'cooperam',
 'de',
 'modo',
 'transparente',
 'para',
 'fornecer',
 'conteúdo',
 'cdn',
 'são',
 'geralmente',
 'implantados',
 'em',
 'vários',
 'locaisos',
 'benefícios',
 'incluem',
 'a',
 'redução',
 'dos',
 'custos',
 'de',
 'largura',
 'de',
 'bandamelhorando',
 'os',
 'tempos',
 'de',
 'carregamento',
 'da',
 'páginaou',
 'aumentando',
 'disponibilidade',
 'global',
 'de',
 'conteúdo',
 'baboo',
 'uma',
 'cdn',
 'é',
 'uma',
 'rede',
 'de',
 'distribuição',
 'de',
 'informação',
 'e',
 'dados',
 'que',
 'permite',
 'fornecer',
 'conteúdo',
 'na',
 'web',
 'de',
 'forma',
 'rápida',
 'para',
 'vários',
 'usuáriosindependentemente',
 'de',
 'onde',
 'essa',
 'pessoa',
 'se',
 'encontrao',
 'que',
 'o',
 'sistema',
 'faz',
 'é',
 'distribuir',
 'um',
 'mesmo',
 'conteúdo',
 'em',
 'diversos',
 'servidorespara',
 'direcionar',
 'esses',
 'dados',
 'de',
 'acordo',
 

In [37]:
port_treino_bigram, vocab_port = padded_everygram_pipeline(2, todas_palavras_port)
esp_treino_bigram, vocab_esp = padded_everygram_pipeline(2, todas_palavras_esp)
ing_treino_bigram, vocab_ing = padded_everygram_pipeline(2, todas_palavras_ing)

In [38]:
next(next(port_treino_bigram))

('<s>',)

## MLE

Modelo por verossimilhança. O objetivo é criar um modelo estatístico de proximidade entre as palavras e letras para identificar o padrão de cada idioma.

In [39]:
modelo_port = MLE(2)
modelo_port.fit(port_treino_bigram, vocab_port)

In [40]:
modelo_esp = MLE(2)
modelo_esp.fit(esp_treino_bigram, vocab_esp)

In [41]:
modelo_ing = MLE(2)
modelo_ing.fit(ing_treino_bigram, vocab_ing)

In [42]:
modelo_port.generate(num_words=9)

['d', 'e', 'o', '</s>', '<s>', 'm', '</s>', 'u', 'i']

### Utilizando o `NgramCounter`

In [43]:
modelo_port.counts[['m']].items()

dict_items([('a', 2180), ('p', 725), ('</s>', 2625), ('o', 890), ('e', 1541), ('i', 303), ('u', 239), ('c', 17), ('é', 86), ('í', 12), ('b', 145), ('m', 46), ('ó', 37), ('l', 25), ('â', 14), ('s', 27), ('á', 38), ('t', 8), ('d', 6), ('j', 7), ('v', 24), ('f', 3), ('y', 12), ('q', 19), ('h', 5), ('ã', 4), ('g', 4), ('ú', 2), ('w', 1), ('n', 8), ('_', 1), ('r', 1)])

## Perplexidade para palavras

Perplexidade é a medida para o teste dos modelos.

In [44]:
texto = "bom dia"
palavras = WhitespaceTokenizer().tokenize(texto)
palavras_fakechar = [list(pad_both_ends(palavra, n = 2)) for palavra in palavras]
palavras_bigrams = [list(bigrams(palavra)) for palavra in palavras_fakechar]
print(palavras_bigrams)
print(modelo_port.perplexity(palavras_bigrams[0]))
print(modelo_port.perplexity(palavras_bigrams[1]))

[[('<s>', 'b'), ('b', 'o'), ('o', 'm'), ('m', '</s>')], [('<s>', 'd'), ('d', 'i'), ('i', 'a'), ('a', '</s>')]]
14.774425459065867
7.994637097583341


In [45]:
def calcular_perplexidade(texto, modelos):
    palavras = WhitespaceTokenizer().tokenize(texto)
    palavras_fakechar = [list(pad_both_ends(palavra, n = 2)) for palavra in palavras]
    palavras_bigrams = [list(bigrams(palavra)) for palavra in palavras_fakechar]
    perplexidades = []
    
    for modelo in modelos:
        palavras_perplexidade = [modelo.perplexity(palavra) for palavra in palavras_bigrams]
        perplexidades.append({"modelo": modelo, "perplexidade": sum(palavras_perplexidade)})
    
    return perplexidades

In [46]:
modelos = [modelo_port, modelo_ing, modelo_esp]

In [47]:
perplexidades = calcular_perplexidade("bom dia", modelos)
minPricedItem = min(perplexidades, key=lambda x:x['perplexidade'])
minPricedItem

{'modelo': <nltk.lm.models.MLE at 0x1fd8d5a4eb0>,
 'perplexidade': 22.76906255664921}

In [48]:
modelos = {modelo_port: "português", modelo_ing: "inglês", modelo_esp: "espanhol"}
key = list(modelos.keys()) [0]
modelos[key]

'português'

## Treinando o modelo

In [49]:
def prever_idioma(texto, modelo_port, modelo_ing, modelo_esp):
    modelos = {modelo_port: "port", modelo_ing: "ing", modelo_esp: "esp"}
    perplexidades = calcular_perplexidade(texto, list(modelos.keys()))
    melhorModelo = min(perplexidades, key=lambda x:x['perplexidade'])
    return modelos[melhorModelo['modelo']]
    

In [51]:
prever_idioma("buenos dias", modelo_port, modelo_ing, modelo_esp)

'esp'

## Avaliando o modelo

### Base de treino

In [52]:
def avaliar_modelo(df, modelo_port, modelo_ing, modelo_esp):
    df['idioma_previsto'] = [prever_idioma(questao, modelo_port, modelo_ing, modelo_esp) for questao in df.questao.values]
    return len(df.query("idioma_previsto == idioma")) / len(df)

In [53]:
avaliar_modelo(df_treino, modelo_port, modelo_ing, modelo_esp)

0.9991666666666666

## Base de teste

In [54]:
df_teste

,questao,idioma
0,moderator note please resist the urge to edit ...,ing
1,tengo unas dudas que me gustaría aclararhe te...,esp
2,si yo intento mostrar la longitud de un array...,esp
3,how do i convert a string to an integer in ja...,ing
4,se eu fizer o hash de senhas antes de armazen...,port
...,...,...
295,quisiera saber cómo detectar el cambio en la ...,esp
296,i know from readingthe microsoft documentatio...,ing
297,creio que a maioria aqui conhece adiferença e...,port
298,en muchas ocasiones he requerido de tomar un ...,esp


In [55]:
avaliar_modelo(df_teste, modelo_port, modelo_ing, modelo_esp)

0.9133333333333333

## Brincando um pouco

In [57]:
prever_idioma("ubatuba", modelo_port, modelo_ing, modelo_esp)

'ing'